In [ ]:
import confmain
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.regularizers import L1L2

from util_module import util_func
from util_module import model_func

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#### **Load X and Y**

In [ ]:
# train_set, val_set, test_set, zpad_length = util_func.get_x_y('../data/ludb_processed/ludb_ii.pickle')

# X_train, y_train = train_set
# X_val, y_val = val_set
# X_test, y_test = test_set

# zpad_length_train, zpad_length_val, zpad_length_test = zpad_length

In [ ]:
# train_set, val_set, test_set, zpad_length = util_func.get_x_y(f'../data/ludb_processed/ludb_{lead}.pickle')

# X_train = util_func.open_pickle(f'../data/temp/{lead}/X_train')
# X_val = util_func.open_pickle(f'../data/temp/{lead}/X_val')
# X_test = util_func.open_pickle(f'../data/temp/{lead}/X_test')
# y_train = util_func.open_pickle(f'../data/temp/{lead}/y_train')
# y_val = util_func.open_pickle(f'../data/temp/{lead}/y_val')
# y_test = util_func.open_pickle(f'../data/temp/{lead}/y_test')
# train_set = X_train, y_train
# val_set = X_val, y_val
# test_set = X_test, y_test

#### **Transfer Learning Model Hafizh**

In [ ]:
# def model_training_wrapper(lead, model_info):
#     train_set, val_set, test_set, zpad_length = util_func.get_x_y(f'../data/ludb_processed/ludb_{lead}.pickle')

#     # X_train = util_func.open_pickle(f'../data/temp/{lead}/X_train')
#     # X_val = util_func.open_pickle(f'../data/temp/{lead}/X_val')
#     # X_test = util_func.open_pickle(f'../data/temp/{lead}/X_test')
#     # y_train = util_func.open_pickle(f'../data/temp/{lead}/y_train')
#     # y_val = util_func.open_pickle(f'../data/temp/{lead}/y_val')
#     # y_test = util_func.open_pickle(f'../data/temp/{lead}/y_test')
#     # train_set = X_train, y_train
#     # val_set = X_val, y_val
#     # test_set = X_test, y_test

#     X_train, y_train = train_set
#     X_val, y_val = val_set
#     X_test, y_test = test_set

#     zpad_length_train, zpad_length_val, zpad_length_test = zpad_length

#     model_hafizh = tf.keras.models.load_model(f'../model/base_model/hafizh.h5')

#     pretrained_layers = model_hafizh.layers[1:-2]

#     for layer in pretrained_layers:
#         layer.trainable = False

#     input_layer = tf.keras.layers.Input(shape=(816, 1))
#     x = tf.keras.layers.Conv1D(8, 3, padding='same')(input_layer)
#     x = tf.keras.layers.LayerNormalization()(x)
#     x = tf.keras.layers.ReLU()(x)

#     for layer in pretrained_layers:
#         x = layer(x)
    
#     x = tf.keras.layers.Dropout(0.3)(x)
#     x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(816, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(1e-4)))(x)
#     x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
#     x = tf.keras.layers.Dropout(0.3)(x)

#     output_layer = tf.keras.layers.Dense(8, activation='softmax')(x)

#     new_model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
#     new_model.compile(optimizer=tf.keras.optimizers.RMSprop(1e-3), loss="categorical_crossentropy", metrics=["accuracy"])

#     start_time = time.time()

#     initial_epochs = 100
#     history_phase_1 = new_model.fit(X_train, y_train, epochs=initial_epochs, batch_size=32, validation_data=(X_val, y_val))

#     # Fine-tune
#     for layer in pretrained_layers[-4:]:
#         layer.trainable = True
#     new_model.compile(optimizer=tf.keras.optimizers.RMSprop(1e-5), loss="categorical_crossentropy", metrics=["accuracy"])

#     fine_tune_epochs = 200
#     total_epochs = initial_epochs + fine_tune_epochs
#     history_phase_2 = new_model.fit(X_train, y_train, epochs=total_epochs, batch_size=32, initial_epoch=history_phase_1.epoch[-1], validation_data=(X_val, y_val))

#     stop_time = time.time()

#     history_combined = {}
#     for key in history_phase_1.history.keys():
#         history_combined[key] = history_phase_1.history[key] + history_phase_2.history[key]

#     model_info['Time elapsed'] = stop_time - start_time

#     model_func.generate_results(new_model, history_combined, model_info, train_set, val_set, test_set, zpad_length)

**Model Hafizh Modified (Without Transfer Learning)**

In [ ]:
def generate_hafizh_modified():
    input_layer = tf.keras.layers.Input(shape=(816, 1))
    x = tf.keras.layers.Conv1D(8, 3, padding='same')(input_layer)
    # x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv1D(16, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.Conv1D(128, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.Conv1D(256, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.Conv1D(512, 3, padding='same', activation='relu')(x)

    # x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(816, return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(1e-4)))(x)
    x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    # x = tf.keras.layers.Dropout(0.3)(x)

    output_layer = tf.keras.layers.Dense(8, activation='softmax')(x)

    new_model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
    new_model.compile(optimizer=tf.keras.optimizers.RMSprop(1e-3), loss="categorical_crossentropy", metrics=["accuracy"])

    return new_model

**Jordan - Complex Model**

In [ ]:
def generate_complex_model(input_shape, output, lr=1e-3, n_layer=1):
    # optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)

    input_layer = tf.keras.layers.Input(shape=input_shape)

    x = tf.keras.layers.Conv1D(8, 5, padding="same")(input_layer)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv1D(16, 5, padding="same")(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv1D(32, 5, padding="same")(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv1D(64, 5, padding="same")(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv1D(128, 5, padding="same")(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv1D(
        256, 7, padding="same", kernel_regularizer=tf.keras.regularizers.l2(1e-4)
    )(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv1D(
        512, 7, padding="same", kernel_regularizer=tf.keras.regularizers.l2(1e-4)
    )(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Dropout(0.3)(x)
    for _ in range(n_layer):
        x = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(
                input_shape[0],
                return_sequences=True,
                kernel_regularizer=tf.keras.regularizers.l2(1e-4),
            )
        )(x)

    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    output_layer = tf.keras.layers.Dense(
        output, activation="softmax", kernel_regularizer=tf.keras.regularizers.l2(1e-4)
    )(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
def model_training_wrapper(lead, model_info):
    train_set, val_set, test_set, zpad_length = util_func.get_x_y(f'../data/ludb_processed/ludb_{lead}.pickle')

    # X_train = util_func.open_pickle(f'../data/temp/{lead}/X_train')
    # X_val = util_func.open_pickle(f'../data/temp/{lead}/X_val')
    # X_test = util_func.open_pickle(f'../data/temp/{lead}/X_test')
    # y_train = util_func.open_pickle(f'../data/temp/{lead}/y_train')
    # y_val = util_func.open_pickle(f'../data/temp/{lead}/y_val')
    # y_test = util_func.open_pickle(f'../data/temp/{lead}/y_test')
    # train_set = X_train, y_train
    # val_set = X_val, y_val
    # test_set = X_test, y_test

    X_train, y_train = train_set
    X_val, y_val = val_set
    X_test, y_test = test_set

    model = generate_complex_model((816, 1), 8, lr=0.00026, n_layer=3)

    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=300, batch_size=32, validation_data=(X_val, y_val))
    stop_time = time.time()

    model_info['Time elapsed'] = stop_time - start_time

    model_func.generate_results(model, history.history, model_info, train_set, val_set, test_set)

In [ ]:
# LEADS = ['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6']
LEADS = ['avf']

for lead in LEADS:
    model_info = {
        'name': f'{lead}-BayesianProposed',
        'lead': lead,
        'batch_size': 32,
        'epochs': 300,
        'n_layer': 3,
        'optimizer': 'RMSprop(lr=0.00026)',
        'additional info': ''
    }

    try:
        model_training_wrapper(lead, model_info)
    
    except:
        print(f'Lead-{lead} failed.')